In [34]:
from transformers import AutoTokenizer, BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
import torch


In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 1)

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# outputs = model(**inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\kjp19/.cache\huggingface\transformers\45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at C:\Users\kjp19/.cache\huggingface\transformers\c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\kjp19/.cache\huggingface\transformers\534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b60

In [37]:
dataset = load_dataset('json', data_files={'train': 'train.jsonl', 'test': 'test.jsonl'})


Using custom data configuration default-930d0c42b71c815f
Reusing dataset json (C:\Users\kjp19\.cache\huggingface\datasets\json\default-930d0c42b71c815f\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


  0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
def preprocess_data(examples):
    # encode a batch of sentences
    encoding = tokenizer(examples["sentence1"], padding="max_length", truncation=True)
    # add labels as a list
    encoding["labels"] = float(examples["label"])
    #encoding['position_ids'] = examples['idx']
    
    return encoding
data_collator = DataCollatorWithPadding(tokenizer)

# tokenize sentences + add labels
encoded_dataset = dataset.map(preprocess_data)
encoded_dataset = encoded_dataset.remove_columns(['word','sentence1','sentence2', 'idx', 'label', 'start1', 'start2', 'end1', 'end2', 'version'])
for k,v in encoded_dataset.items():
     print(k, v.shape)
# turn into PyTorch dataset
encoded_dataset.set_format("torch")



small_train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = encoded_dataset["test"].shuffle(seed=42).select(range(100))


  0%|          | 0/5000 [00:00<?, ?ex/s]

  0%|          | 0/638 [00:00<?, ?ex/s]

train (5000, 4)
test (638, 4)


In [48]:
print(small_train_dataset)
print(small_train_dataset['input_ids'])

for i in small_train_dataset['labels']: 
    print(i)




Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100
})
tensor([[  101,  2260,  2132,  ...,     0,     0,     0],
        [  101,  1996, 15114,  ...,     0,     0,     0],
        [  101,  7374,  1037,  ...,     0,     0,     0],
        ...,
        [  101,  2202,  2048,  ...,     0,     0,     0],
        [  101,  1045,  2134,  ...,     0,     0,     0],
        [  101,  2323,  2057,  ...,     0,     0,     0]])
tensor(1.)
tensor(0.)
tensor(0.)
tensor(1.)
tensor(1.)
tensor(0.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(0.)
tensor(1.)
tensor(1.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(1.)
tensor(1.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.)
tens

In [ ]:
training_args = TrainingArguments(
    output_dir=".",
    evaluation_strategy="epoch",
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=5
)

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset,data_collator = data_collator)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 100
  Num Epochs = 5
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 50


Epoch,Training Loss,Validation Loss


In [50]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [51]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


{'eval_loss': 0.2632523477077484,
 'eval_accuracy': 0.48,
 'eval_runtime': 296.7979,
 'eval_samples_per_second': 0.337,
 'eval_steps_per_second': 0.044}